In [2]:
import torch
a=[13.5,12.5,11.5]
a=torch.tensor(a)
a.round()



tensor([14., 12., 12.])

In [2]:
torch.tensor([-2.5000 , -1.5000,   -0.5000,    0.5000 ,   1.5000  ,  2.5000]).round()

tensor([-2., -2., -0.,  0.,  2.,  2.])

In [3]:
X=torch.tensor([[[1,2,3,4],[1,2,3,4]],[[1,2,3,4],[1,2,3,4]]])
Y=torch.tensor([1])
X*Y

tensor([[[1, 2, 3, 4],
         [1, 2, 3, 4]],

        [[1, 2, 3, 4],
         [1, 2, 3, 4]]])

In [13]:
x =torch.tensor([[[1,2,3,4]]])
x.shape
x.expand(1,4,4)
for i in range(x.shape[2]):
    print(i)

0
1
2
3


In [56]:
out_scale=torch.tensor(0.234)
in_scale=torch.rand(1,1,384)
channel_nums=384
SCALE=torch.rand(1,1,384)
x_q=torch.rand(16,197,384)
x_q_sum=x_q.sum(dim=-1)
std_x_q =  torch.sqrt(#std标准差也就是加入alpha后每一行的标准差，
                channel_nums * (x_q**2).sum(dim=-1) - x_q.sum(dim=-1)**2)
print(x_q_sum.shape)
for i in range(x_q.shape[2]):#x_q:[1,197,384]
    x_q[:,:,i]=(x_q[:,:,i]*channel_nums-x_q_sum)/std_x_q
    
for i in range(x_q.shape[1]):
    x_q[:,i,:]=x_q[:,i,:]*in_scale

beta=torch.rand(384)



X_q=torch.rand(1,197,384)
print(X_q.shape)
X_Mean=X_q.sum(dim=-1).unsqueeze(-1)*torch.ones(384)
print(X_Mean.shape,X_Mean)


Bias=torch.rand(384)
Bias=Bias.reshape(1,-1,1)
print(Bias.shape)
Bias=Bias*torch.ones(197)
print(Bias.transpose(-1,-2).shape,Bias)

torch.Size([16, 197])
torch.Size([1, 197, 384])
torch.Size([1, 197, 384]) tensor([[[189.5131, 189.5131, 189.5131,  ..., 189.5131, 189.5131, 189.5131],
         [185.2674, 185.2674, 185.2674,  ..., 185.2674, 185.2674, 185.2674],
         [202.7715, 202.7715, 202.7715,  ..., 202.7715, 202.7715, 202.7715],
         ...,
         [187.6707, 187.6707, 187.6707,  ..., 187.6707, 187.6707, 187.6707],
         [192.0286, 192.0286, 192.0286,  ..., 192.0286, 192.0286, 192.0286],
         [187.3412, 187.3412, 187.3412,  ..., 187.3412, 187.3412, 187.3412]]])
torch.Size([1, 384, 1])
torch.Size([1, 197, 384]) tensor([[[0.3004, 0.3004, 0.3004,  ..., 0.3004, 0.3004, 0.3004],
         [0.9279, 0.9279, 0.9279,  ..., 0.9279, 0.9279, 0.9279],
         [0.1231, 0.1231, 0.1231,  ..., 0.1231, 0.1231, 0.1231],
         ...,
         [0.4158, 0.4158, 0.4158,  ..., 0.4158, 0.4158, 0.4158],
         [0.9842, 0.9842, 0.9842,  ..., 0.9842, 0.9842, 0.9842],
         [0.4404, 0.4404, 0.4404,  ..., 0.4404, 0.4404, 0.4

In [5]:
import torch
batch=16
x_q=torch.rand(batch,197,384)#构造输入
in_scale=torch.rand(1,1,384)
out_scale=torch.rand(1)
channel_nums=x_q.shape[-1]#获取通道数
print(x_q[0,:,:])

x_q=(x_q/in_scale).round()
M1=x_q.sum(dim=-1)
print(M1.shape)
print(M1[:,1].shape)
print(M1[:,1].reshape(16,-1,1).shape)
#然后让每一行的点减去对应行的sum
std_x_q = torch.sqrt(#std标准差也就是加入alpha后每一行的标准差，
                channel_nums * (x_q**2).sum(dim=-1) - x_q.sum(dim=-1)**2)#[1,197]
for i in range(M1.shape[-1]):#对于每一行来说，都是[16,384]维，对于M1的每一行来说，都是[16]维
    x_q[:,i,:]=(x_q[:,i,:]-M1[:,i].reshape(batch,-1))/std_x_q[:,i].reshape(batch,-1)


Gama=torch.rand(1,384)
Beta=torch.rand(1,384)
for i in range(x_q.shape[1]):#对于每一行来说
    x_q[:,i,:]=(x_q[:,i,:]*Gama+Beta).round()
# x=torch.tensor([[[1,2,3],[4,5,6]]])
# M1=x.sum(dim=-1)
# print(x)
# print(M1)
# print(x[:,1,:]-M1[:,1])
a=torch.pow(2,torch.tensor(32))
topk=(1,)
print(topk)

tensor([[0.4387, 0.7669, 0.2106,  ..., 0.8591, 0.9018, 0.4752],
        [0.4014, 0.7293, 0.1607,  ..., 0.2853, 0.2908, 0.7871],
        [0.2099, 0.5888, 0.8882,  ..., 0.1855, 0.9568, 0.5268],
        ...,
        [0.8973, 0.6117, 0.1119,  ..., 0.9328, 0.6377, 0.8109],
        [0.0457, 0.5935, 0.6763,  ..., 0.4758, 0.0696, 0.9165],
        [0.2328, 0.5865, 0.9871,  ..., 0.3980, 0.9654, 0.4865]])
torch.Size([16, 197])
torch.Size([16])
torch.Size([16, 1, 1])
(1,)


In [6]:
import torch
#研究acuracy是如何计算的
output=torch.rand(16,1000)#16个batch，1000个类的概率
topk=(1,5)
maxk=max(topk)
_, pred = output.topk(maxk, 1, True, True)